In [1]:
import os

%pwd

In [2]:
%pwd

'c:\\Users\\learn\\Documents\\KidneyDisease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\learn\\Documents\\KidneyDisease-Classification-Deep-Learning-Project'

In [5]:
import os
import mlflow

# Manually set environment variables
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/rakibrohan54/KidneyDisease-Classification-Deep-Learning-Project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'rakibrohan54'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '68d2276306699e6c4c999c53f8494162ae6ca912'

# Set the experiment name (this should match an existing experiment or it will create one)
mlflow.set_experiment('KidneyDiseaseClassification2')

# Start MLflow experiment
with mlflow.start_run():
    mlflow.log_param('parameter name', 'value')
    mlflow.log_metric('metric name', 1)


2024/10/04 10:54:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run puzzled-cub-267 at: https://dagshub.com/rakibrohan54/KidneyDisease-Classification-Deep-Learning-Project.mlflow/#/experiments/2/runs/996da97c93be4fc1b539fec79056fed2.
2024/10/04 10:54:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/rakibrohan54/KidneyDisease-Classification-Deep-Learning-Project.mlflow/#/experiments/2.


In [6]:

import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/rakibrohan54/KidneyDisease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.


c:\Users\learn\Documents\KidneyDisease-Classification-Deep-Learning-Project\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 42s 4s/step - accuracy: 0.8102 - loss: 1.6897


2024/10/04 10:55:41 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/04 10:56:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/10/04 10:57:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 7
Created version '7' of model 'VGG16Model'.
2024/10/04 10:57:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run delicate-smelt-219 at: https://dagshub.com/rakibrohan54/KidneyDisease-Classification-Deep-Learning-Project.mlflow/#/experiments/2/runs/a4c6d14a89bf402e98bd5dec6c88938f.
2024/10/04 10:57:07 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/rakibrohan54/KidneyDisease-Classificatio

In [14]:
print(os.environ['MLFLOW_TRACKING_URI'])
print(os.environ['MLFLOW_TRACKING_USERNAME'])

https://dagshub.com/rakibrohan54/KidneyDisease-Classification-Deep-Learning-Project.mlflow
rakibrohan54
